In [23]:
from google_bigquery import bq_to_csv, csv_to_bq
import pandas as pd
import json
import yaml
import re
import glob
from datetime import date
import unidecode
import numpy as np
import string
from tqdm import tqdm

In [24]:
dataset_id = 'maxi_prices'
table_id ='price'
folder = str(date.today())
store = '/loblaws/'
path = '/Users/simonprudhomme/Desktop/maxi/'
path+folder

'/Users/simonprudhomme/Desktop/maxi/2020-04-01'

In [25]:
all_files = glob.glob(path+folder + "/*.csv")

li = []
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

data = pd.concat(li, axis=0, sort=True,ignore_index=True)
data['variation'] = np.nan
data['is_max'] = 0
data['is_similar']=0
print(data.shape)
data = data.drop_duplicates(subset=['source','productSKU','productCategory','productPrice'])
print(data.shape)

(13556, 15)
(13556, 15)


In [26]:
duplicates = data['productSKU'].value_counts().to_frame()
duplicate_value = list(duplicates[duplicates['productSKU']==2]['productSKU'].index)

In [27]:
for i in tqdm(duplicate_value):
    max_a = data[data['productSKU']==i]['productPrice'].max()
    min_b = data[data['productSKU']==i]['productPrice'].min()
    
    variation_decrease = abs(max_a-min_b)/(max_a)
    variation_increase = abs(min_b-max_a)/(min_b)
    variation = abs(min_b - max_a)/((min_b + max_a)/2)
    
    data.loc[((data['productSKU']==i)&(data['productPrice']==max_a)),'variation'] = variation_increase
    data.loc[((data['productSKU']==i)&(data['productPrice']==max_a)),'is_max']=1
    data.loc[(data['productSKU']==i),'is_similar']=1
    data.loc[(data['productSKU']==i),'is_similar']=1

100%|██████████| 2512/2512 [00:25<00:00, 99.49it/s] 


In [28]:
print(data[data['source']=='loblaws']['variation'].mean())
print(data[data['source']=='maxi']['variation'].mean())
print(data[data['source']=='loblaws']['variation'].mean()-data[data['source']=='maxi']['variation'].mean())
data.groupby(by=['source','productCategory'])['variation'].mean().to_frame().unstack()

0.35159165387762437
0.19414414713283698
0.15744750674478739


variation                                                   \
productCategory    bakery     dairy      deli    frozen      meat   pantry   
source                                                                       
loblaws          0.162905  0.203507  0.184914  0.149374  0.065717  0.37876   
maxi             0.074566  0.209785  0.041493  0.208390  0.062146  0.25511   

                            
productCategory vegetables  
source                      
loblaws           1.289920  
maxi              0.183026

In [29]:
data[data['is_similar']==1].sort_values(by='productSKU', ascending=True)

,date,dealBadge,productBrand,productCategory,productName,productPrice,productPricePerGram,productSKU,productSales,productURL,productUnit,source,variation,is_max,is_similar
6356,2020-04-01,NaN,President's Choice,deli,Goat Cheese,5.99,4.28,20000584,NaN,https://www.loblaws.ca/p/20000584_EA,100g,loblaws,0.054577,1,1
1965,2020-04-01,NaN,President's Choice,deli,Goat Cheese,5.68,4.06,20000584,NaN,https://www.maxi.ca/p/20000584_EA,100g,maxi,NaN,0,1
7615,2020-04-01,NaN,PC Blue Menu,pantry,Sun-Dried Tomato & Herb Seasoning Blend,4.99,5.87,20000963,NaN,https://www.loblaws.ca/p/20000963_EA,100g,loblaws,0.165888,1,1
2148,2020-04-01,NaN,PC Blue Menu,pantry,Sun-Dried Tomato & Herb Seasoning Blend,4.28,5.04,20000963,NaN,https://www.maxi.ca/p/20000963_EA,100g,maxi,NaN,0,1
5662,2020-04-01,NaN,NaN,vegetables,Medjool Dates,14.99,1.65,20001686,NaN,https://www.loblaws.ca/p/20001686_EA,100g,loblaws,0.154854,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5468,2020-04-01,multi,NaN,vegetables,Avocado,1.69,NaN,21097012001,NaN,https://www.loblaws.ca/p/21097012001_EA,NaN,loblaws,NaN,0,1
5083,2020-04-01,NaN,NaN,vegetables,Organic Broccoli Crowns,2.50,6.57,21098393001,NaN,https://www.maxi.ca/p/21098393001_KG,1kg,maxi,NaN,0,1
5537,2020-04-01,NaN,NaN,vegetables,Organic Broccoli Crowns,3.76,9.90,21098393001,NaN,https://www.loblaws.ca/p/21098393001_KG,1kg,loblaws,0.504000,1,1
5479,2020-04-01,NaN,No Name,vegetables,Limes,5.49,0.61,21124368001,NaN,https://www.loblaws.ca/p/21124368001_EA,100g,loblaws,0.379397,1,1


In [30]:
data.sort_values(by='productPrice',ascending=False)

,date,dealBadge,productBrand,productCategory,productName,productPrice,productPricePerGram,productSKU,productSales,productURL,productUnit,source,variation,is_max,is_similar
6375,2020-04-01,NaN,Mastro,deli,Salami with Prosciutto (Thin Sliced),3490.00,NaN,20315926,NaN,https://www.loblaws.ca/p/20315926_KG,NaN,loblaws,NaN,0,0
5401,2020-04-01,NaN,NaN,vegetables,Limes (Case),72.00,1.33,20040489001,NaN,https://www.loblaws.ca/p/20040489001_C01,1ea,loblaws,104.882353,1,1
5310,2020-04-01,NaN,NaN,vegetables,Red Seedless Grapes (Case),64.00,7.87,20159199001,NaN,https://www.loblaws.ca/p/20159199001_C01,1kg,loblaws,NaN,0,0
6616,2020-04-01,NaN,NaN,deli,GRUYERE KINGCUTS,59.90,9.98,20435155,NaN,https://www.loblaws.ca/p/20435155_KG,1kg,loblaws,NaN,0,0
11688,2020-04-01,NaN,President's Choice,meat,Rack of Lamb Lollipop,56.19,55.09,21057132,NaN,https://www.loblaws.ca/p/21057132_KG,1kg,loblaws,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2451,2020-04-01,NaN,No Name Club Pack,pantry,Chicken Vegetable Flavour Oriental Noodles & S...,0.33,0.39,20305832,NaN,https://www.maxi.ca/p/20305832_EA,100g,maxi,NaN,0,0
5147,2020-04-01,NaN,NaN,vegetables,Chinese Long Beans (Bodie),0.33,0.66,20162508001,NaN,https://www.maxi.ca/p/20162508001_KG,100g,maxi,NaN,0,1
5629,2020-04-01,sale,NaN,vegetables,King Oyster Mushrooms,0.32,1.30,20860588,0.39,https://www.loblaws.ca/p/20860588_KG,100g,loblaws,NaN,0,0
2585,2020-04-01,sale,Maggi,pantry,"Noodles, Masala",0.28,0.40,20802664,0.10,https://www.maxi.ca/p/20802664_EA,100g,maxi,NaN,0,0


In [31]:
data['original'] =np.where(data['productSales'].isnull()==False, data['productPrice']+data['productSales'], data['productPrice'])
data['discount'] =  abs(data['original']-data['productPrice'])/(data['original'])
del data['original']
data.sort_values(by='discount', ascending=False).head(20)

,date,dealBadge,productBrand,productCategory,productName,productPrice,productPricePerGram,productSKU,productSales,productURL,productUnit,source,variation,is_max,is_similar,discount
9666,2020-04-01,sale,Crown,pantry,Crown Hearts of Palm Whole,0.99,0.25,21178970,2.00,https://www.loblaws.ca/p/21178970_EA,100mL,loblaws,NaN,0,0,0.668896
11558,2020-04-01,sale,NaN,meat,"Top Sirloin Steak, Cap Off",11.19,11.00,20973326,22.43,https://www.loblaws.ca/p/20973326_KG,1kg,loblaws,NaN,0,0,0.667162
371,2020-04-01,sale,Five Alive,frozen,Passionate Peachy Citrus,0.50,0.17,20418326003,0.88,https://www.maxi.ca/p/20418326003_EA,100mL,maxi,NaN,0,1,0.637681
245,2020-04-01,sale,Five Alive,frozen,Citrus,0.50,0.17,20418326001,0.88,https://www.maxi.ca/p/20418326001_EA,100mL,maxi,NaN,0,0,0.637681
5459,2020-04-01,multi,Farmer's Market,vegetables,Mini Carrots,2.29,0.67,20117370001,4.00,https://www.loblaws.ca/p/20117370001_EA,100g,loblaws,0.156566,1,1,0.635930
5287,2020-04-01,sale,NaN,vegetables,"Mango Ataulfo, Case (Case)",8.88,NaN,21047848,14.11,https://www.maxi.ca/p/21047848_C01,NaN,maxi,NaN,0,1,0.613745
2070,2020-04-01,multi,Schneider,deli,Lunch Mate Stackers Ham,3.28,3.64,20770779,5.00,https://www.maxi.ca/p/20770779_EA,100g,maxi,NaN,0,0,0.603865
11614,2020-04-01,sale,NaN,meat,Top Sirloin Roast Beef,15.40,11.00,20973338,21.60,https://www.loblaws.ca/p/20973338_KG,1kg,loblaws,NaN,0,0,0.583784
59,2020-04-01,sale,Minute Maid,frozen,Pink Lemonade,0.50,0.17,20418457003,0.68,https://www.maxi.ca/p/20418457003_EA,100mL,maxi,NaN,0,0,0.576271
433,2020-04-01,sale,Minute Maid,frozen,Lemonade,0.50,0.17,20418457002,0.68,https://www.maxi.ca/p/20418457002_EA,100mL,maxi,NaN,0,0,0.576271


In [32]:
filename_clean = str(date.today())+'_final_clean.csv'
data.to_csv(path+folder+'/'+filename_clean,index=False)

In [33]:
data.shape

(13556, 16)

In [34]:
csv_to_bq(path+folder+'/'+filename_clean, dataset_id, table_id, overwrite=False, auto=True)

Loaded 13556 rows into maxi_prices:price.
